<a href="https://colab.research.google.com/github/yuukienomoto/mahjong/blob/main/mj_ltGBM_feat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gzip
import xml.etree.ElementTree as ET
import glob
from google.colab import drive
import time
import numpy as np
import random
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

print("--- ステップ1: Google Driveのマウント ---")
try:
    drive.mount('/content/drive')
except:
    pass

# 2. フォルダパス指定
log_folder_path = '/content/drive/MyDrive/mjlog_pf4-20_n28/'
file_pattern = log_folder_path + '*.mjlog'
log_files = glob.glob(file_pattern)

if not log_files:
    print(f"エラー: {log_folder_path} に .mjlog ファイルが見つかりません。")
else:
    print(f"全 {len(log_files)} 個のファイルが見つかりました。")

    final_dataset = []

    # --- 牌コード解析ヘルパー関数 (0-33の「種類」コードに対応) ---
    def _parse_tile_type(tile_type_code):
        if 0 <= tile_type_code <= 8:     return 'manzu', tile_type_code + 1
        elif 9 <= tile_type_code <= 17: return 'pinzu', (tile_type_code - 9) + 1
        elif 18 <= tile_type_code <= 26: return 'souzu', (tile_type_code - 18) + 1
        elif 27 <= tile_type_code <= 33: return 'jihai', (tile_type_code - 27) + 1
        else: return 'unknown', -1

    # ★★★【新機能】ドラ表示牌から「ドラ」を計算する関数 ★★★
    def get_actual_dora(indicator_type_code):
        """0-33のドラ表示牌コードから、実際のドラ(0-33)を返す"""
        tile_type, tile_num = _parse_tile_type(indicator_type_code)

        if tile_type == 'unknown': return -1

        if tile_type == 'jihai':
            if tile_num <= 4: # E(27) -> S(28) ... N(30) -> E(27)
                return 27 + (tile_num % 4)
            else: # 白(31) -> 發(32) ... 中(33) -> 白(31)
                return 31 + ((tile_num - 1) % 3)

        # 数牌 (9 -> 1)
        if tile_num == 9:
            return indicator_type_code - 8 # (例: 9m(8) -> 1m(0))
        # 数牌 (1-8 -> 2-9)
        else:
            return indicator_type_code + 1


    # ★★★【ロジック修正版】11次元の本格的な特徴量計算関数 ★★★
    def calculate_features(game_state, player_index, physical_code):

        # ★ 物理コード(0-135)を、種類コード(0-33)に変換
        tile_type_code = physical_code // 4

        features = []

        # --- 0. 公開情報リストの作成 ---
        all_public_tiles = []
        all_public_tiles.extend(game_state['hands'][player_index])
        for discard_pile in game_state['discards']:
            all_public_tiles.extend(discard_pile)

        # 1. 牌の種類 (0-33)
        features.append(tile_type_code)

        # 2. 見え枚数 (Visible Count) (バグ修正済み)
        visible_count = all_public_tiles.count(tile_type_code)
        features.append(visible_count)

        # 3. 相手リーチ (Opponent Reach)
        opponent_reach = any(game_state['reach'][i] for i in range(4) if i != player_index)
        features.append(1 if opponent_reach else 0)

        # (特徴量 4, 5, 6 は前回と同じ)
        in_opponent_discard = any(tile_type_code in game_state['discards'][i] for i in range(4) if i != player_index)
        features.append(1 if in_opponent_discard else 0) # 4
        recent_discards_1 = [t for p, t in game_state['history'][-4:]]
        features.append(1 if tile_type_code in recent_discards_1 else 0) # 5
        recent_discards_2 = [t for p, t in game_state['history'][-8:]]
        features.append(1 if tile_type_code in recent_discards_2 else 0) # 6

        # --- 7, 8, 9 (スジ・カベ・順子) ---
        tile_type, tile_num = _parse_tile_type(tile_type_code)

        # 7. 順子の作成の可否 (Shuntsu)
        shuntsu_potential = 0
        if tile_type in ['manzu', 'pinzu', 'souzu']:
            if tile_num in [1, 9]: shuntsu_potential = 1
            elif tile_num in [2, 8]: shuntsu_potential = 2
            elif tile_num in [3, 4, 5, 6, 7]: shuntsu_potential = 3
        features.append(shuntsu_potential)

        # 8. スジ (Suji)
        is_suji = False
        if tile_type in ['manzu', 'pinzu', 'souzu']:
            suji_pairs = []
            if tile_num in [1, 4, 7]: suji_pairs = [1, 4, 7]
            if tile_num in [2, 5, 8]: suji_pairs = [2, 5, 8]
            if tile_num in [3, 6, 9]: suji_pairs = [3, 6, 9]
            opponent_discards_parsed = []
            for i in range(4):
                if i != player_index:
                    for discarded_code in game_state['discards'][i]:
                        d_type, d_num = _parse_tile_type(discarded_code)
                        if d_type == tile_type:
                            opponent_discards_parsed.append(d_num)
            if tile_num in [4, 5, 6]:
                pair1, pair2 = suji_pairs[0], suji_pairs[2]
                if pair1 in opponent_discards_parsed and pair2 in opponent_discards_parsed:
                    is_suji = True
        features.append(1 if is_suji else 0)

        # 9. カベ (Kabe) (バグ修正済み)
        is_kabe_tile = 0
        if tile_type in ['manzu', 'pinzu', 'souzu']:
            suit_start_code = (tile_type_code // 9) * 9
            if tile_num >= 2 and all_public_tiles.count(suit_start_code + (tile_num - 2)) == 4:
                is_kabe_tile = 1
            if tile_num <= 8 and all_public_tiles.count(suit_start_code + (tile_num)) == 4:
                is_kabe_tile = 1
        features.append(1 if is_kabe_tile else 0)

        # 10. 巡目 (Turn Number)
        turn_number = len(game_state['history']) // 4
        features.append(turn_number)

        # 11. ★★★【新特徴量】ドラの数 ★★★
        dora_count = 0
        # a) 赤ドラかどうか (物理コード 16, 52, 88)
        if physical_code in [16, 52, 88]:
            dora_count += 1
        # b) 表示牌の次の牌かどうか
        actual_dora_code = get_actual_dora(game_state['dora_indicator'])
        if tile_type_code == actual_dora_code:
            dora_count += 1
        features.append(dora_count)

        return features

--- ステップ1: Google Driveのマウント ---
Mounted at /content/drive
全 3181 個のファイルが見つかりました。


In [ ]:
   # --- ステップ2: 全ファイル (3181個) の再解析 ---
print("\n--- ステップ2: 全ファイルの再解析を開始します (11次元・ドラ機能搭載) ---")
start_time = time.time()
files_to_process = log_files

for i, file_path in enumerate(files_to_process):

        if (i + 1) % 100 == 0:
            print(f"  処理中... {i+1} / {len(files_to_process)} ファイル ( {(time.time() - start_time):.1f} 秒経過)")

        try:
            with gzip.open(file_path, 'rt', encoding='utf-8') as f:
                tree = ET.parse(f)
                root = tree.getroot()

            game_state = {
                'hands': [[], [], [], []], 'discards': [[], [], [], []],
                'reach': [False, False, False, False], 'history': [],
                'dora_indicator': -1 # ★ドラ表示牌
            }
            last_discard_event = None

            for tag in root:
                if tag.tag == 'INIT':
                    if last_discard_event: final_dataset.append(last_discard_event)

                    # ★【新機能】ドラ表示牌をseedから取得
                    dora_indicator_phys_code = int(tag.attrib.get('seed').split(',')[4])
                    dora_indicator_type_code = dora_indicator_phys_code // 4

                    game_state = {
                        'hands': [[], [], [], []], 'discards': [[], [], [], []],
                        'reach': [False, False, False, False], 'history': [],
                        'dora_indicator': dora_indicator_type_code # ★記憶
                    }
                    last_discard_event = None
                    hai_strings = [tag.attrib.get(h) for h in ['hai0', 'hai1', 'hai2', 'hai3']]
                    for p_idx in range(4):
                        if hai_strings[p_idx]:
                            game_state['hands'][p_idx] = [int(s) // 4 for s in hai_strings[p_idx].split(',')]

                draw_prefix = tag.tag[0]
                if draw_prefix in {'T', 'U', 'V', 'W'}:
                    try:
                        player_index = {'T': 0, 'U': 1, 'V': 2, 'W': 3}[draw_prefix]
                        physical_code = int(tag.tag[1:])
                        tile_type_code = physical_code // 4
                        game_state['hands'][player_index].append(tile_type_code)
                    except (ValueError, KeyError): pass

                discard_prefix = tag.tag[0]
                if discard_prefix in {'D', 'E', 'F', 'G'}:
                    try:
                        if last_discard_event: final_dataset.append(last_discard_event)
                        player_index = {'D': 0, 'E': 1, 'F': 2, 'G': 3}[discard_prefix]
                        physical_code = int(tag.tag[1:])

                        # ★ 11次元の特徴量を計算 (物理コードを渡す)
                        features_X = calculate_features(game_state, player_index, physical_code)
                        last_discard_event = {'X': features_X, 'y': 0}

                        tile_type_code = physical_code // 4 # 種類コードに変換
                        if tile_type_code in game_state['hands'][player_index]:
                            game_state['hands'][player_index].remove(tile_type_code)
                        game_state['discards'][player_index].append(tile_type_code)
                        game_state['history'].append((player_index, tile_type_code))
                    except (ValueError, KeyError): pass

                if tag.tag == 'REACH':
                    if tag.attrib.get('step') == '1':
                         game_state['reach'][int(tag.attrib.get('who'))] = True

                if tag.tag == 'AGARI':
                    winner = int(tag.attrib.get('who'))
                    loser = tag.attrib.get('fromWho')
                    if loser and winner != int(loser):
                        score = int(tag.attrib.get('ten').split(',')[1])
                        if last_discard_event:
                            last_discard_event['y'] = score
                            final_dataset.append(last_discard_event)
                            last_discard_event = None

        except Exception as e:
            print(f"  ファイル {file_path} の処理中にエラー: {e}")

end_time = time.time()
print("\n-------------------------")
print(f"全 {len(files_to_process)} ファイルの解析が完了しました。")
print(f"処理時間: {end_time - start_time:.2f} 秒")
print(f"作成された総データセット件数: {len(final_dataset)}")


--- ステップ2: 全ファイルの再解析を開始します (11次元・ドラ機能搭載) ---
  処理中... 100 / 3181 ファイル ( 69.5 秒経過)
  処理中... 200 / 3181 ファイル ( 70.8 秒経過)
  処理中... 300 / 3181 ファイル ( 72.1 秒経過)
  処理中... 400 / 3181 ファイル ( 73.5 秒経過)
  処理中... 500 / 3181 ファイル ( 75.3 秒経過)
  処理中... 600 / 3181 ファイル ( 77.0 秒経過)
  処理中... 700 / 3181 ファイル ( 79.2 秒経過)
  処理中... 800 / 3181 ファイル ( 81.6 秒経過)
  処理中... 900 / 3181 ファイル ( 83.6 秒経過)
  処理中... 1000 / 3181 ファイル ( 85.8 秒経過)
  処理中... 1100 / 3181 ファイル ( 86.9 秒経過)
  処理中... 1200 / 3181 ファイル ( 87.8 秒経過)
  処理中... 1300 / 3181 ファイル ( 88.7 秒経過)
  処理中... 1400 / 3181 ファイル ( 90.0 秒経過)
  処理中... 1500 / 3181 ファイル ( 91.1 秒経過)
  処理中... 1600 / 3181 ファイル ( 92.1 秒経過)
  処理中... 1700 / 3181 ファイル ( 93.6 秒経過)
  処理中... 1800 / 3181 ファイル ( 94.7 秒経過)
  処理中... 1900 / 3181 ファイル ( 96.0 秒経過)
  処理中... 2000 / 3181 ファイル ( 97.5 秒経過)
  処理中... 2100 / 3181 ファイル ( 99.1 秒経過)
  処理中... 2200 / 3181 ファイル ( 101.5 秒経過)
  処理中... 2300 / 3181 ファイル ( 103.2 秒経過)
  処理中... 2400 / 3181 ファイル ( 105.1 秒経過)
  処理中... 2500 / 3181 ファイル ( 106.7 秒経過)
  処理中... 

In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import random
import time
import pandas as pd

print("\n--- ステップ3: LightGBM モデルの再学習 (11次元・ドラ機能搭載) ---")

if (not 'final_dataset') or (len(final_dataset) == 0):
    print("エラー: 'final_dataset' が見つかりません。")
else:
    # 1. 136万件の全データを X と y に分離
    print(f"全 {len(final_dataset)} 件のデータセットを X と y に分離しています...")

    feature_names = [
        '1. 牌コード (Tile Code)', '2. 見え枚数 (Visible Count)',
        '3. 相手リーチ (Opponent Reach)', '4. 相手の河にあるか (In Opponent Discard)',
        '5. 1巡以内 (Discarded 1-Turn)', '6. 2巡以内 (Discarded 2-Turns)',
        '7. 順子 (Shuntsu)', '8. スジ (Suji)', '9. カベ (Kabe)',
        '10. 巡目 (Turn Number)', '11. ドラ (Dora Count)' # ★新特徴量
    ]

    X_list = [item['X'] for item in final_dataset]
    y = np.array([item['y'] for item in final_dataset])

    X = pd.DataFrame(X_list, columns=feature_names)

    # ★「牌コード」を「カテゴリ変数」としてAIに教える
    X['1. 牌コード (Tile Code)'] = X['1. 牌コード (Tile Code)'].astype('category')
    print("「牌コード」をカテゴリ変数として設定しました。")

    print(f"\n特徴量Xの形状 (全データ): {X.shape}") # (..., 11) になっているはず
    print(f"教師値yの形状 (全データ): {y.shape}")

    # 2. 訓練データとテストデータに分割
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"訓練データ数: {len(X_train)}, テストデータ数: {len(X_test)}")

    # 3. LightGBM モデルの定義
    lgbm_model = lgb.LGBMRegressor(
        objective='regression_l2',
        is_unbalance=True,
        n_estimators=200,
        learning_rate=0.1,
        n_jobs=-1
    )

    # 4. モデルの学習
    print("\nLightGBMモデルの学習を開始します...")
    start_train_time = time.time()

    lgbm_model.fit(
        X_train,
        y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(10)],
        categorical_feature=['1. 牌コード (Tile Code)']
    )

    end_train_time = time.time()
    print(f"学習が完了しました！ (学習時間: {end_train_time - start_time:.2f} 秒)")

    # 5. モデルの「全体的な精度」を評価
    print("\n--- モデル全体の精度評価 (最終モデル) ---")
    y_pred = lgbm_model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print(f"モデルの予測ズレ (RMSE): {rmse:.2f} 点")
    print(f"  (参考: 6次元モデルのズレは約 637.25 点でした)")


--- ステップ3: LightGBM モデルの再学習 (11次元・ドラ機能搭載) ---
全 1365231 件のデータセットを X と y に分離しています...
「牌コード」をカテゴリ変数として設定しました。

特徴量Xの形状 (全データ): (1365231, 11)
教師値yの形状 (全データ): (1365231,)
訓練データ数: 1092184, テストデータ数: 273047

LightGBMモデルの学習を開始します...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.085903 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 78
[LightGBM] [Info] Number of data points in the train set: 1092184, number of used features: 11
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 53.288732
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[43]	valid_0's rmse: 629.475	valid_0's l2: 396239
学習が完了しました！ (学習時間: 276.83 秒)

--- モデル全体の精度評価 (最終モデル) ---
モデルの予測

In [ ]:
import pandas as pd
try:
    print(f"--- LightGBM 特徴量の重要度 (最終・11次元モデル) ---")
    importances = lgbm_model.feature_importances_
    feature_importance_df = pd.DataFrame({
        'Feature': lgbm_model.feature_name_,
        'Importance Score': importances
    })
    feature_importance_df = feature_importance_df.sort_values(by='Importance Score', ascending=False)
    print(feature_importance_df.to_string())
except NameError:
    print("\nエラー: 'lgbm_model' が見つかりません。")

--- LightGBM 特徴量の重要度 (最終・11次元モデル) ---
                              Feature  Importance Score
0                 1._牌コード_(Tile_Code)               364
9                10._巡目_(Turn_Number)               307
1             2._見え枚数_(Visible_Count)               147
2           3._相手リーチ_(Opponent_Reach)                95
3   4._相手の河にあるか_(In_Opponent_Discard)                81
5         6._2巡以内_(Discarded_2-Turns)                70
8                        9._カベ_(Kabe)                68
7                        8._スジ_(Suji)                62
6                     7._順子_(Shuntsu)                40
4          5._1巡以内_(Discarded_1-Turn)                29
10                11._ドラ_(Dora_Count)                27
